In [108]:
import pandas as pd
from sklearn.linear_model import ElasticNetCV
import itertools
import statsmodels.api as sm
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_regression
from sklearn.pipeline import Pipeline
from sklearn import linear_model
import numpy as np
from numpy import arange
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
import sys
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    warnings.filterwarnings('ignore')
    
from sklearn.metrics import mean_squared_error    


In [ ]:
def read_data():
    df = pd.read_csv("https://raw.githubusercontent.com/thistleknot/Python-Stock/master/data/raw/states.csv").set_index('States')
    return(df)

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# ------------------------------------
# file: zca.py
# date: Thu May 21 15:47 2015
# author:
# Maarten Versteegh
# github.com/mwv
# maartenversteegh AT gmail DOT com
#
# Licensed under GPLv3
# ------------------------------------
"""zca: ZCA whitening with a sklearn-like interface

"""

from __future__ import division

import numpy as np
from scipy import linalg

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import check_array, as_float_array

class ZCA(BaseEstimator, TransformerMixin):
    def __init__(self, regularization=1e-6, copy=False):
        self.regularization = regularization
        self.copy = copy

    def fit(self, X, y=None):
        """Compute the mean, whitening and dewhitening matrices.

        Parameters
        ----------
        X : array-like with shape [n_samples, n_features]
            The data used to compute the mean, whitening and dewhitening
            matrices.
        """
        X = check_array(X, accept_sparse=None, copy=self.copy,
                        ensure_2d=True)
        X = as_float_array(X, copy=self.copy)
        self.mean_ = X.mean(axis=0)
        X_ = X - self.mean_
        cov = np.dot(X_.T, X_) / (X_.shape[0]-1)
        U, S, _ = linalg.svd(cov)
        s = np.sqrt(S.clip(self.regularization))
        s_inv = np.diag(1./s)
        s = np.diag(s)
        self.whiten_ = np.dot(np.dot(U, s_inv), U.T)
        self.dewhiten_ = np.dot(np.dot(U, s), U.T)
        return self

    def transform(self, X, y=None, copy=None):
        """Perform ZCA whitening

        Parameters
        ----------
        X : array-like with shape [n_samples, n_features]
            The data to whiten along the features axis.
        """
        check_is_fitted(self, 'mean_')
        X = as_float_array(X, copy=self.copy)
        return np.dot(X - self.mean_, self.whiten_.T)

    def inverse_transform(self, X, copy=None):
        """Undo the ZCA transform and rotate back to the original
        representation

        Parameters
        ----------
        X : array-like with shape [n_samples, n_features]
            The data to rotate back.
        """
        check_is_fitted(self, 'mean_')
        X = as_float_array(X, copy=self.copy)
        return np.dot(X, self.dewhiten_) + self.mean_


In [ ]:
independent = 'Poverty'

In [ ]:
y = read_data()[[independent]]
X = read_data()[(read_data().columns).difference([independent]).values]

In [ ]:
estimators = []
estimators.append(('standardize', ZCA()))
estimators.append(('ElasticNetCV', ElasticNetCV(cv=10, random_state=0)))
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
ratios = arange(0, 1, 0.01)
alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
#estimators.append(('ElasticNetCV', ElasticNetCV(l1_ratio=ratios, alphas=alphas, cv=cv, n_jobs=-1)))
model = Pipeline(estimators)

model.fit(X, y)
#print('alpha: %f' % model.alphas)
#print('l1_ratio_: %f' % model.l1_ratio)


In [ ]:
model.score(X,y)

In [ ]:
model.get_params()

In [ ]:
results = cross_val_score(model, X, y, cv=cv)

In [ ]:
results

In [ ]:
#X, y = make_regression(n_features=2, random_state=0)
#regr = ElasticNetCV(cv=10, random_state=0)
#regr.fit(X, y)
#print(regr.alpha_)
#print(regr.intercept_)
#print(regr.predict(X))

In [104]:


rmse_enet = mean_squared_error(y, model.predict(X), squared=True)
rmse_enet

1.1771762311443557

In [ ]:
plt.scatter(model.predict(X),y)

In [ ]:
#chosen_model = ElasticNet(alpha=model.named_steps['ElasticNetCV'].alpha_, l1_ratio=model.named_steps['ElasticNetCV'].l1_ratio)
#chosen_model.fit(X,y)

In [ ]:
#plt.scatter(chosen_model.predict(X),y)

In [ ]:
#pd.DataFrame(model.predict(X)-chosen_model.predict(X)).hist()

In [ ]:
pd.DataFrame(np.array(pd.DataFrame([model.named_steps['ElasticNetCV'].coef_,]))*np.array(zca.fit_transform(X)),columns=X.columns).set_index(y.index).sum(axis=1).sort_values(axis=0)

In [ ]:
pd.DataFrame(zca.fit_transform(X[filtered]),columns=filtered),model.named_steps['ElasticNetCV'].coef_

In [ ]:
filtered = X.columns[model.named_steps['ElasticNetCV'].coef_!=0]

#from statsmodels.formula.api import ols
sm_model = sm.OLS(y, sm.add_constant(X[filtered])).fit()
sm_model.summary()




In [ ]:
#model.named_steps['standardize'].fit_transform(X)*model.named_steps['ElasticNetCV'].coef_

#model.named_steps['standardize'].fit_transform(X)
#pd.DataFrame(zca.fit_transform(X[filtered]),columns=filtered),model.named_steps['ElasticNetCV'].alpha_
#pd.DataFrame(zca.fit_transform(X[filtered]),columns=filtered),model.named_steps['ElasticNetCV'].l1_ratio


In [ ]:
#pd.DataFrame([model.named_steps['standardize'].coef_],columns=X.columns)

In [ ]:
#pd.DataFrame([model.named_steps['ElasticNetCV'].coef_],columns=X.columns)

In [116]:
def fit_linear_reg(X,Y):
    #Fit linear regression model and return RSS and R squared values
    model_k = linear_model.LinearRegression(fit_intercept = True)
    model_k.fit(X,Y)
    RSS = mean_squared_error(Y,model_k.predict(X)) * len(Y)
    R_squared = model_k.score(X,Y)
    return RSS, R_squared
    
#Importing tqdm for the progress bar
from tqdm import tnrange, tqdm_notebook

#Initialization variables

k = 10
RSS_list, R_squared_list, feature_list = [],[], []
numb_features = []

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

#Looping over k = 1 to k = 11 features in X
for k in tnrange(1,len(X.columns) + 1, desc = 'Loop...'):

    #Looping over all possible combinations: from 11 choose k
    for combo in itertools.combinations(X.columns,k):
        zca = ZCA()
        
        #linear regression with CV
        model = ElasticNet(alpha=0, l1_ratio=0)
        model.fit(pd.DataFrame(zca.fit_transform(X[list(combo)]),columns=X[list(combo)].columns),y)
        #tmp_result = model.predict(fit_linear_reg(pd.DataFrame(zca.fit_transform(X[list(combo)]),columns=X[list(combo)].columns),y)   #Store temp result 
        tmp_result = model.predict(zca.fit_transform(X[list(combo)]))  #Store temp result 
        RSS_list.append(tmp_result[0])                  #Append lists
        R_squared_list.append(tmp_result[1])
        feature_list.append(combo)
        numb_features.append(len(combo))   

#Store in DataFrame
df_cv = pd.DataFrame({'numb_features': numb_features,'RSS': RSS_list, 'R_squared':R_squared_list,'features':feature_list})    

Loop...:   0%|          | 0/9 [00:00<?, ?it/s]

In [119]:
import matplotlib.pyplot as plt
df_cv

,numb_features,RSS,R_squared,features
0,1,12.910386,13.848102,"(Crime,)"
1,1,13.554547,13.353191,"(Doctors,)"
2,1,15.394385,8.025266,"(Income,)"
3,1,15.420799,12.818735,"(Infant Mort,)"
4,1,12.645080,12.396401,"(Population,)"
...,...,...,...,...
506,8,16.144712,8.699962,"(Crime, Doctors, Income, Population, Traf Deat..."
507,8,15.462325,12.373729,"(Crime, Doctors, Infant Mort, Population, Traf..."
508,8,16.120499,8.809363,"(Crime, Income, Infant Mort, Population, Traf ..."
509,8,16.088543,8.726147,"(Doctors, Income, Infant Mort, Population, Tra..."


In [103]:
zca = ZCA()
sm_model_zca = sm.OLS(y, sm.add_constant(pd.DataFrame(zca.fit_transform(X[filtered]),columns=filtered)).set_index(y.index)).fit()
#sm_model_zca = sm.OLS(y, sm.add_constant(pd.DataFrame(zca.fit_transform(X[filtered]),columns=filtered)).set_index(y.index)).fit()

sm_model_zca.summary()



rmse_linear = mean_squared_error(y, sm_model_zca.predict(sm.add_constant(zca.fit_transform(X[filtered]))), squared=True)
rmse_linear

1.047869360518312